In [ ]:
# Install required packages (runs automatically in Colab, fast no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

# OpenQASM 3 und das Qiskit SDK

{/*
  DO NOT EDIT THIS CELL!!!
  This cell's content is generated automatically by a script. Anything you add
  here will be removed next time the notebook is run. To add new content, create
  a new cell before or after this one.
*/}

<details>
<summary><b>Package versions</b></summary>

The code on this page was developed using the following requirements.
We recommend using these versions or newer.

```
qiskit[all]~=2.3.0
```
</details>
Das Qiskit SDK bietet einige Tools zur Konvertierung zwischen OpenQASM-Darstellungen von Quantenprogrammen und der QuantumCircuit-Klasse. Beachten Sie, dass sich diese Tools noch in einer explorativen Entwicklungsphase befinden und sich weiterentwickeln werden, da die Unterstützung von Qiskit für durch OpenQASM 3 ausgedrückte dynamische Circuit-Fähigkeiten zunimmt.

> **Note:** Diese Funktion befindet sich noch in der explorativen Phase. Daher ist es wahrscheinlich, dass sich die Syntax und die Fähigkeiten weiterentwickeln werden.

## Importieren eines OpenQASM 3-Programms in Qiskit
Sie müssen das Paket `qiskit_qasm3_import` installieren, um diese Funktion zu verwenden. Installieren Sie es mit folgendem Befehl.

In [1]:
import qiskit.qasm3

program = """
    OPENQASM 3.0;
    include "stdgates.inc";

    input float[64] a;
    qubit[3] q;
    bit[2] mid;
    bit[3] out;

    let aliased = q[0:1];

    gate my_gate(a) c, t {
      gphase(a / 2);
      ry(a) c;
      cx c, t;
    }
    gate my_phase(a) c {
      ctrl @ inv @ gphase(a) c;
    }

    my_gate(a * 2) aliased[0], q[{1, 2}][0];
    measure q[0] -> mid[0];
    measure q[1] -> mid[1];

    while (mid == "00") {
      reset q[0];
      reset q[1];
      my_gate(a) q[0], q[1];
      my_phase(a - pi/2) q[1];
      mid[0] = measure q[0];
      mid[1] = measure q[1];
    }

    if (mid[0]) {
      let inner_alias = q[{0, 1}];
      reset inner_alias;
    }

    out = measure q;
"""
circuit = qiskit.qasm3.loads(program)
circuit.draw("mpl")

<Image src="../docs/images/guides/interoperate-qiskit-qasm3/extracted-outputs/e805197c-51fb-4216-8d24-ae26633d29bc-0.svg" alt="Output of the previous code cell" />

Derzeit sind zwei High-Level-Funktionen für den Import von OpenQASM 3 in Qiskit verfügbar. Diese Funktionen sind `load()`, die einen Dateinamen akzeptiert, und `loads()`, die das Programm selbst als String akzeptiert:

In [2]:
from qiskit import QuantumCircuit
from qiskit.qasm3 import dumps

qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.measure_all()

dumps(qc)

'OPENQASM 3.0;\ninclude "stdgates.inc";\nbit[2] meas;\nqubit[2] q;\nh q[0];\ncx q[0], q[1];\nbarrier q[0], q[1];\nmeas[0] = measure q[0];\nmeas[1] = measure q[1];\n'

In diesem Beispiel definieren wir ein Quantenprogramm mit OpenQASM 3 und verwenden `loads()`, um es direkt in einen QuantumCircuit zu konvertieren:

In [3]:
from qiskit import QuantumCircuit
from qiskit.qasm3 import dump

qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.measure_all()

f = open("my_file.txt", "w")
dump(qc, f)
f.close()

![Output of the previous code cell](../docs/images/guides/interoperate-qiskit-qasm3/extracted-outputs/e805197c-51fb-4216-8d24-ae26633d29bc-0.svg)

## Exportieren nach OpenQASM 3
Sie können Qiskit-Code nach OpenQASM 3 mit `dumps()`, das in einen String exportiert, oder `dump()`, das in eine Datei exportiert, exportieren.

### Beispiel mit `dumps()`